In [1]:
import openfoamparser_mai as Ofpp
import os
import numpy as np
import dgl
import networkx as nx
import torch
import matplotlib.pyplot as plt
from bokeh.palettes import Spectral

In [2]:
data_V = []
target_V = []

## 1. Препроцессинг данных

In [3]:
for directory in sorted(os.listdir('data_wage\\data_wage\\low_dim\\')):
    data_dir = None
    target_dir = None
    
    for file in sorted(os.listdir('data_wage\\data_wage\\low_dim\\' + directory)):
        if file not in ['constant', 'system', '0']:
            if data_dir is not None:
                data_dir = np.append(data_dir, np.array([Ofpp.parse_internal_field('data_wage\\data_wage\\low_dim\\' \
                                                                               + directory + '\\' + file + '\\U')]), axis=0)
                target_dir = np.append(target_dir, np.array([Ofpp.parse_internal_field('data_wage\\data_wage\\high_dim\\' \
                                                                               + directory + '\\' + file + '\\U')]), axis=0)
            else:
                data_dir = np.array([Ofpp.parse_internal_field('data_wage\\data_wage\\low_dim\\' \
                                                             + directory + '\\' + file + '\\U')])
                target_dir = np.array([Ofpp.parse_internal_field('data_wage\\data_wage\\high_dim\\' \
                                                             + directory + '\\' + file + '\\U')])
        
    data_V.append(data_dir)
    target_V.append(target_dir)

In [4]:
import torch

In [5]:
low_num_nodes = 75
def create_low_dim_graph(path, features):
    mesh = Ofpp.FoamMesh(path)

    neighbours = []

    for i in range(low_num_nodes):
        for j in list(filter(lambda x: 0 <= x < low_num_nodes, mesh.cell_neighbour_cells(i))):
            neighbours.append((i, j))

    g = dgl.graph(neighbours)
    g.ndata["attr"] = torch.from_numpy(features).float()
    return g

In [6]:
DIR_PATH = 'data_wage\\data_wage\\low_dim\\'
data = []

for i, directory in enumerate(sorted(os.listdir(DIR_PATH))):
    for j in range(10):
        data.append(create_low_dim_graph(DIR_PATH + directory + "\\", data_V[i][j][:, :3]))

In [7]:
high_num_nodes = 4800

def create_high_dim_graph(path, features):
    mesh = Ofpp.FoamMesh(path)

    neighbours = []

    for i in range(high_num_nodes):
        for j in list(filter(lambda x: 0 <= x < high_num_nodes, mesh.cell_neighbour_cells(i))):
            neighbours.append((i, j))

    g = dgl.graph(neighbours)
    g.ndata["attr"] = torch.from_numpy(features).float()
    return g

In [8]:
from tqdm import tqdm

In [9]:
DIR_PATH = 'data_wage\\data_wage\\high_dim\\'
target = []

for i, directory in tqdm(enumerate(sorted(os.listdir(DIR_PATH)))):
    for j in range(10):
        target.append(create_high_dim_graph(DIR_PATH + directory + "\\", target_V[i][j][:, :3]))

100it [02:23,  1.44s/it]


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=42)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [11]:
y_train[0].ndata['attr']

tensor([[ 5.8687e+00, -7.4313e-15,  0.0000e+00],
        [ 5.8687e+00, -2.2237e-14,  0.0000e+00],
        [ 5.8687e+00, -2.6675e-14,  0.0000e+00],
        ...,
        [ 5.8687e+00,  2.1250e-13,  0.0000e+00],
        [ 5.8687e+00,  1.6029e-13,  0.0000e+00],
        [ 5.8687e+00, -1.7198e-14,  0.0000e+00]])

In [56]:
import dgl

In [13]:
edge_index = y_train[1].edges()

In [14]:
edge_index_75 = X_train[1].edges()

## 2. GAN training

In [15]:
import wandb
import dgl

wandb.login()

wandb: Currently logged in as: ildarnikitin20. Use `wandb login --relogin` to force relogin


True

In [16]:
wandb.init()

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
from dgl.nn.pytorch import GraphConv
from tqdm import tqdm
import torch.nn.functional as F


device = torch.device('cuda')
    
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1 = GraphConv(3, 256)
        self.conv2 = GraphConv(256, 4800 * 3)

    def forward(self, g):
        h = self.conv1(g, g.ndata['attr'])
        h = F.relu(h)
        h = self.conv2(g, h).view(-1, 4800, 3)
        h = h.mean(dim=0)
#         print(h.shape)
        new_g = dgl.graph(edge_index).to(device)
        new_g.ndata['attr'] = h.to(device)
        return new_g

# Define the discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = GraphConv(3, 16)
        self.conv2 = GraphConv(16, 1)

    def forward(self, g):
        h = g.ndata['attr']
        h = F.relu(self.conv1(g, h))
        h = self.conv2(g, h)
        h = h.mean(dim=0)

        h = torch.sigmoid(h)
        return h
    
discriminator = Discriminator().to(device)
generator = Generator().to(device)

# Define the training loop
criterion = nn.BCELoss()
criterion2 = nn.MSELoss()

optimizer_g = optim.Adam(generator.parameters(), lr=0.0005)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0005)

for epoch in range(15):
    all_d_loss = 0
    all_g_loss = 0

    for i in tqdm(range(len(X_train))):
        # Train the discriminator
        discriminator.zero_grad()

        real_data = y_train[i]
        real_label = torch.ones_like(discriminator(real_data.to(device)))
        fake_graph_features = torch.randn(75, 3)
        fake_graph = dgl.graph(edge_index_75).to(device)
        fake_graph.ndata['attr'] = fake_graph_features.to(device)
        fake_data = generator(fake_graph)
        fake_label = torch.zeros_like(discriminator(fake_data))

        d_loss_real = criterion(discriminator(real_data.to(device)), real_label)
        d_loss_fake = criterion(discriminator(fake_data.to(device)), fake_label)
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_d.step()
        
        all_d_loss += d_loss 
    all_d_loss /= (i + 1)
    
    for i in tqdm(range(len(X_train))):
        # Train the generator
        generator.zero_grad()
        
        fake_data = generator(X_train[i].to(device))
        g_loss = criterion(discriminator(fake_data), real_label)
        p_loss = criterion2(y_train[i].ndata['attr'].to(device), fake_data.ndata['attr'].to(device))
        g_loss += p_loss
        g_loss.backward()
        optimizer_g.step()
        
        all_g_loss += g_loss 
    all_g_loss /= (i + 1)

        # Output training stats
    if epoch % 1 == 0:
        print('d_loss: {:.4f}, g_loss: {:.4f}'.format(all_d_loss, all_g_loss))
        record = {
            'd_loss': all_d_loss,
            'g_loss': all_g_loss
        }
        wandb.log(record)



100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:13<00:00, 66.36it/s]


d_loss: 0.9355, g_loss: 4.7665


100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:13<00:00, 64.66it/s]


d_loss: 0.6813, g_loss: 0.2174


100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:13<00:00, 68.62it/s]


...

d_loss: 0.0042, g_loss: 0.0153


100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:13<00:00, 66.78it/s]


d_loss: 0.0026, g_loss: 0.0148


100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:13<00:00, 68.76it/s]


d_loss: 0.0017, g_loss: 0.0144


## 3. Генерация графов повышенного разрешения

In [87]:
result = []
for i in tqdm(range(len(X_test))):
    result.append(generator(X_test[i].to(device)).ndata['attr'].cpu().detach().numpy())

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 78.48it/s]


In [88]:
result = torch.from_numpy(np.array(result).reshape((100, 4800, 3)))

In [89]:
targets = []
for g in y_test:
    targets.append(g.ndata['attr'].cpu().detach().numpy())
    
targets = torch.from_numpy(np.array(targets).reshape((100, 4800, 3)))

In [92]:
from torchmetrics import MeanAbsolutePercentageError
mean_abs_percentage_error = MeanAbsolutePercentageError()
mean_abs_percentage_error(targets[:, :, :2], result[:, :, :2])

tensor(0.4958)